In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
requests.packages.urllib3.disable_warnings(
    requests.packages.urllib3.exceptions.InsecureRequestWarning)

In [ ]:
df = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep='\t')

In [ ]:
f_df = df.loc[df['genres'].str.contains('Fantasy', na=False)]

In [ ]:
def check_genres(example):
    includes = ['Fantasy']
    excludes = ['Adult', 'Documentary', 'Talk-Show', 'Sport', 'Game-Show', 'Musical', 'Music', 'News', 'Reality-TV', 'Family']

    try:
        if any([x in example for x in includes]) and all([x not in example for x in excludes]):
            print(example)
            return True
    except:
        pass

    return False

In [ ]:
fin_df = df.loc[df['genres'].apply(check_genres)]

In [ ]:
fin_df.to_pickle('/content/drive/MyDrive/AI/150k_with_excludes')

In [ ]:
fin_df = pd.read_pickle('/content/drive/MyDrive/AI/150k_with_excludes')

In [ ]:
plot_url = "https://www.imdb.com/title/{}/plotsummary/"
summaries_class = "ipc-metadata-list__item"

keywords_url = "https://www.imdb.com/title/{}/keywords"
tag_class = "soda sodavote"

headers = [{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'},
           {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Safari/605.1.15'},
           {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}]

In [ ]:
def clean(input_text):
    input_text = input_text.replace('\n', ' ')
    return input_text.replace('\t', '')

In [ ]:
def get_summaries(imdb_id):
    # Send a GET request to the URL
    response = requests.get(plot_url.format(imdb_id), headers=headers[1], verify=False)

    # Create a BeautifulSoup object from the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    elements = soup.find_all(class_=summaries_class)[:-2] # trunicate service elements

    clean_elements = []
    for el in elements:

        try:
            el.find("span").decompose() # remove author tag from summary if exists
        except AttributeError:
            pass

        clean_elements.append(el.get_text())

    return clean_elements

In [ ]:
def filter_tag(tag):
    tag_name = tag['data-item-keyword']
    relevant_text = tag.find(class_="interesting-count-text").text.strip()
    stop_words = ['sex', 'nudity', 'erotic', 'male', 'female', 'f word']

    if any(stop_word in tag_name for stop_word in stop_words):
        return False

    if relevant_text.startswith("0") or relevant_text == "Is this relevant?":
        return False

    return tag_name

In [ ]:
def get_tags(imdb_id):
    response = requests.get(keywords_url.format(imdb_id), headers=headers[1], verify=False)
    soup = BeautifulSoup(response.content, 'html.parser')
    elements = soup.find_all(class_=tag_class)

    filtered_tags = []

    for tag in elements:
        if a:=filter_tag(tag):
            filtered_tags.append(a)

    return filtered_tags

In [ ]:
def add_to_tsv(imdb_id):
    tags = get_tags(imdb_id)
    time.sleep(0.25) # avoid spamming imdb servers
    summs = get_summaries(imdb_id)

    if len(tags) == 0 or len(summs) == 0:
        return False

    str_tags = " | ".join(tags)

    for summ in summs:
        with open("key_and_plot.tsv", "a", encoding="utf-8") as kp_file:
            kp_file.write(f"{imdb_id}\t{str_tags}\t{clean(summ)}\n")

    return len(summs)

In [ ]:
count_n = 0

while True:
    try:
        for index, row in fin_df[count_n:].iterrows():
            imdb_id = row["tconst"]

            if l:=add_to_tsv(imdb_id):
                print(count_n, f"added {l} summs", imdb_id)
            else:
                print(count_n, "no plots or no keywords for", imdb_id)
            count_n += 1
            time.sleep(0.25)
    except:
        pass